<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

<h2>Part 1</h2>

<h3>Dataframe of the postal code of each neighborhood</h3>

<p> First we need to import packages</p>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

<p> We will be using Beatiful Soup to pull the data from the wikipedia page and tabulate it in useful manner. BS4 package has all what we need from pulling html from website and putting into pandas dataframe. But first we need to set up couple things</p>

<p> We target the source url that we want to use;then we specify source content to read with bea</p>

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.content,'lxml')

<p>And we specifically select our table by its html tag and css class name</p> 

In [3]:
table = soup.find_all('table', class_="wikitable sortable")[0]

<p>We create dataframe by reading the table </p> 

In [4]:
df = pd.read_html(str(table))[0]

<p>We set first tow to be the headers </p> 

In [5]:
headers = df.iloc[0]
df = pd.DataFrame(df.values[1:], columns=headers)

<p> Ignoring not assigned borough cells </p>  

In [6]:
df = df[df.Borough != 'Not assigned']

<p> Group based on postal code, combine neighborhoods in one cell </p>

In [7]:
df = df.groupby('Postcode').agg({'Postcode':'first', 'Borough':'first', 'Neighbourhood':', '.join})

<p> Replace not assigned neighborhood with borough name</p>

In [8]:
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

<p> Making index run from 0 consequentailly </p>

In [9]:
df = df.reset_index(drop=True)
df.head(10)

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough
9,M1N,"Birch Cliff, Cliffside West",Scarborough


In [10]:
df.shape

(103, 3)

<h2> Part 2 </h2>

<h3>Latitude and the longitude coordinates of each neighborhood</h3>

In [11]:
# The code was removed by Watson Studio for sharing.

In [12]:
df_geocode = pd.read_csv(body)
df_geocode.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [13]:
df_geocode.shape

(103, 3)

In [14]:
hoods = pd.concat([df,df_geocode], axis=1)
hoods = hoods.drop(['Postal Code'], axis=1) 

In [15]:
hoods.head(10)

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,M1B,"Rouge, Malvern",Scarborough,43.806686,-79.194353
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough,43.784535,-79.160497
2,M1E,"Guildwood, Morningside, West Hill",Scarborough,43.763573,-79.188711
3,M1G,Woburn,Scarborough,43.770992,-79.216917
4,M1H,Cedarbrae,Scarborough,43.773136,-79.239476
5,M1J,Scarborough Village,Scarborough,43.744734,-79.239476
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",Scarborough,43.727929,-79.262029
7,M1L,"Clairlea, Golden Mile, Oakridge",Scarborough,43.711112,-79.284577
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",Scarborough,43.716316,-79.239476
9,M1N,"Birch Cliff, Cliffside West",Scarborough,43.692657,-79.264848


<h2> Part 3 </h2>

<h3>Explore and cluster the neighborhoods in Toronto</h3>

<h4>1. Download and Explore Dataset</h4>

In [16]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.14 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.66 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  24.61 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  47.27 MB/s


In [17]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.23 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  32.80 MB/s


In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.653963, -79.387207.


<p>Create a map of New York with neighborhoods superimposed on top </p>

In [25]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, label in zip(hoods['Latitude'], hoods['Longitude'], hoods['Borough'], hoods['Neighbourhood']):
    label = '{}, {}'.format(hoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [26]:
CLIENT_ID = 'KO1JMGG15LOU4RWXABMYJ5FYFQBVLVAPJA2FPCLWODSFE0XZ' # your Foursquare ID
CLIENT_SECRET = 'AGUQVGUBKTF1ZMZIOWW5RRY1RGWJUUBQDW4FZMT5KJ1TBJ0N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KO1JMGG15LOU4RWXABMYJ5FYFQBVLVAPJA2FPCLWODSFE0XZ
CLIENT_SECRET:AGUQVGUBKTF1ZMZIOWW5RRY1RGWJUUBQDW4FZMT5KJ1TBJ0N


In [27]:
hoods.loc[0, 'Neighbourhood']

'Rouge, Malvern'

In [28]:
neighborhood_latitude = hoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = hoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = hoods.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


In [29]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KO1JMGG15LOU4RWXABMYJ5FYFQBVLVAPJA2FPCLWODSFE0XZ&client_secret=AGUQVGUBKTF1ZMZIOWW5RRY1RGWJUUBQDW4FZMT5KJ1TBJ0N&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c1fea9a4434b91ed84bc8a9'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4bb6b9446edc76b0d771311c-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16e941735',
         'name': 'Fast Food Restaurant',
         'pluralName': 'Fast Food Restaurants',
         'primary': True,
         'shortName': 'Fast Food'}],
       'id': '4bb6b9446edc76b0d771311c',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Morningside & Sheppard',
        'distance': 387,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.80744841934756,
          'ln

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


<h4>2. Explore Neighborhoods in Toronto</h4>

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
hoods_venues = getNearbyVenues(names=hoods['Neighbourhood'],
                                   latitudes=hoods['Latitude'],
                                   longitudes=hoods['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [39]:
print(hoods_venues.shape)
hoods_venues.head(10)

(2247, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
5,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764042,-79.193371,Rental Car Location
7,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop


In [40]:
hoods_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
"Alderwood, Long Branch",10,10,10,10,10,10
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",27,27,27,27,27,27
Berczy Park,54,54,54,54,54,54


In [41]:
print('There are {} uniques categories.'.format(len(hoods_venues['Venue Category'].unique())))

There are 274 uniques categories.


<h4>3. Analyze Each Neighborhood</h4>

In [60]:
# one hot encoding
hoods_onehot = pd.get_dummies(hoods_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hoods_onehot['Neighborhood'] = hoods_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hoods_onehot.columns[-1]] + list(hoods_onehot.columns[:-1])
hoods_onehot = hoods_onehot[fixed_columns]

hoods_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
hoods_onehot.shape

(2247, 274)

In [62]:
hoods_grouped = hoods_onehot.groupby('Neighborhood').mean().reset_index()
hoods_grouped.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
5,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.058824,0.0,0.0,0.00,0.0,0.00
6,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
7,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
8,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00
9,"Birch Cliff, Cliffside West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.00


In [67]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [114]:
# create a new dataframe
hoods_venues_sorted = pd.DataFrame(columns=columns)
hoods_venues_sorted['Neighborhood'] = hoods_grouped['Neighborhood']

for ind in np.arange(hoods_grouped.shape[0]):
    hoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hoods_grouped.iloc[ind, :], num_top_venues)

hoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Asian Restaurant,Bakery,Hotel,Clothing Store,Restaurant
1,Agincourt,Breakfast Spot,Skating Rink,Clothing Store,Lounge,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Fried Chicken Joint,Coffee Shop,Sandwich Place,Pharmacy,Fast Food Restaurant,Beer Store,Gluten-free Restaurant,Dance Studio
4,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Coffee Shop,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Gym,Airport Service
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Sushi Restaurant,Deli / Bodega,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Bridal Shop,Sandwich Place,Diner,Fast Food Restaurant
6,Bayview Village,Bank,Chinese Restaurant,Café,Japanese Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dim Sum Restaurant
7,"Bedford Park, Lawrence Manor East",Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Greek Restaurant,Sandwich Place,Juice Bar,Liquor Store,Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Cheese Shop,Café,Pub,Seafood Restaurant,Farmers Market,Steakhouse,Italian Restaurant
9,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


<h4>4. Cluster Neighborhoods</h4>

In [115]:
# set number of clusters
kclusters = 5

hoods_grouped_clustering = hoods_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hoods_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_.shape

(99,)

In [116]:
hoods_merged = hoods

df_kmeans = kmeans.labels_

# add clustering labels
hoods_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hoods_merged = hoods_merged.join(hoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

hoods_merged.head() # check the last columns!

ValueError: Length of values does not match length of index

In [117]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [118]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hoods_merged['Latitude'], hoods_merged['Longitude'], hoods_merged['Neighbourhood'], hoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

IndexError: list index out of range